In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Multi_class Calssification  Sequential


In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

X_train, X_valid, X_test = X_train / 255.0, X_valid / 255.0, X_test / 255.0

In [ ]:
class_names = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [ ]:
y_train.shape

In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis("off")
        plt.title(class_names[y_train[index]])
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=[28, 28]),
        tf.keras.layers.Dense(300, activation="relu"),
        tf.keras.layers.Dense(100, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)
model.summary()

In [ ]:
# extra code – this cell is equivalent to the previous cell
model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    optimizer=tf.keras.optimizers.SGD(),
    metrics=[tf.keras.metrics.sparse_categorical_accuracy],
)

In [ ]:
history = model.fit(X_train, y_train, epochs=9, validation_data=(X_valid, y_valid))

In [ ]:
pd.DataFrame(history.history).plot(
    figsize=(8, 5),
    xlim=[0, 29],
    ylim=[0, 1],
    grid=True,
    xlabel="Epoch",
    style=["r--", "r--.", "b-", "b-*"],
)
plt.legend(loc="lower left")  # extra code
plt.show()

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_probabilities = model.predict(X_test[90:115])
predictions = y_probabilities.argmax(axis=-1)
np.array(class_names)[predictions]

y_probabilities

In [ ]:
actual = y_test[90:115]
np.array(class_names)[actual]

# Regression Sequential

In [ ]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42
)

X_train.__len__()

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])

model_reg = tf.keras.Sequential(
    [
        norm_layer,
        tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.Dense(50, activation="relu"),
        tf.keras.layers.Dense(1),
    ]
)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.003)

model_reg.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

norm_layer.adapt(X_train)
history = model_reg.fit(
    X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), batch_size=80
)

In [ ]:
mse_test, rmse_test = model_reg.evaluate(X_test, y_test)
mse_test, rmse_test

In [ ]:
pd.DataFrame(history.history)

# RandomForest same data, test

In [15]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

model.fit(X_train, y_train)

preds = model.predict(X_test[15:25])


preds, y_test[15:25]



(array([1.63525  , 2.02629  , 2.03884  , 1.41593  , 0.8754   , 4.0508919,
        3.783993 , 1.78361  , 4.0753917, 1.76857  ]),
 array([1.362  , 1.784  , 1.875  , 1.398  , 1.375  , 4.25   , 4.056  ,
        1.388  , 5.00001, 1.621  ]))

# Wide and Deep Neural net for California Housing

In [ ]:
# Wide means input layer directly connected to output(concat)
# Deep means 2+ hidden layer connected to output

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(4)

# creating layers
input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
norm_layer = tf.keras.layers.Normalization()
dense_1 = tf.keras.layers.Dense(50, activation="relu")
dense_2 = tf.keras.layers.Dense(50, activation="relu")
concat = tf.keras.layers.Concatenate()
output = tf.keras.layers.Dense(1)

# creating the interconnections of layers

normilized = norm_layer(input_)
hidden1 = dense_1(normilized)
hidden2 = dense_2(hidden1)
concat = concat([normilized, hidden2])
output = output(concat)


model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

In [ ]:
model.compile(loss="mse", optimizer="adam", metrics=["RootMeanSquaredError"])

In [ ]:
norm_layer.adapt(X_train)

history = model.fit(X_train, y_train, epochs=15, validation_data=(X_valid, y_valid))

model.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(history.history)

# Wide and Deep net with multiple inputs for Wide and Deep* (Regression example)

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(99)

input_wide = tf.keras.layers.Input(name="wide_input", shape=[5])
input_deep = tf.keras.layers.Input(name="deep_input", shape=[6])

norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden_1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden_2 = tf.keras.layers.Dense(30, activation="relu")(hidden_1)
concat_ = tf.keras.layers.concatenate([norm_wide, hidden_2])

output = tf.keras.layers.Dense(1)(concat_)

model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

model.summary()

In [ ]:
model.compile(loss="mse", optimizer="adam", metrics=["RootMeanSquaredError"])

In [ ]:
X_train_wide, X_train_deep, X_valid_wide, X_valid_deep, X_test_wide, X_test_deep = (
    X_train[:, :5],
    X_train[:, 2:],
    X_valid[:, :5],
    X_valid[:, 2:],
    X_test[:, :5],
    X_test[:, 2:],
)

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit(
    (X_train_wide, X_train_deep),
    y_train,
    epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), y_valid),
)

In [ ]:
model.evaluate((X_test_wide, X_test_deep), y_test)

# Wide and Deep net with multiple inputs(wide and deep input) and Multiple outputs for Regularization (Regression example)

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(100)

input_wide = tf.keras.layers.Input(name="wide_input", shape=[5])
input_deep = tf.keras.layers.Input(name="deep_input", shape=[6])

norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden_1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden_2 = tf.keras.layers.Dense(30, activation="relu")(hidden_1)
concat_ = tf.keras.layers.concatenate([norm_wide, hidden_2])

output = tf.keras.layers.Dense(1, name="output_main")(concat_)
auxiliary_output = tf.keras.layers.Dense(1, name="output_aux")(hidden_2)

model = tf.keras.Model(
    inputs=[input_wide, input_deep], outputs=[output, auxiliary_output]
)

model.compile(
    loss=("mse", "mse"),
    loss_weights=(0.9, 0.1),
    optimizer="adam",
    metrics=["RootMeanSquaredError"],
)

model.summary()

In [ ]:
X_train_wide, X_train_deep, X_valid_wide, X_valid_deep, X_test_wide, X_test_deep = (
    X_train[:, :5],
    X_train[:, 2:],
    X_valid[:, :5],
    X_valid[:, 2:],
    X_test[:, :5],
    X_test[:, 2:],
)

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)


history = model.fit(
    (X_train_wide, X_train_deep),
    (y_train, y_train),
    epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)),
)

In [ ]:
model.evaluate((X_test_wide, X_test_deep), (y_test, y_test), return_dict=True)

In [ ]:
X_new_wide = X_test_wide[10:20]
X_new_deep = X_test_deep[10:20]

y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

y_pred_aux

# Subclassing API for dynamic models (Regression example)

In [ ]:
class WideDeepModel(tf.keras.Model):
    def __init__(self, units = 30 , activation = 'relu', **kwargs):
        super().__init__(**kwargs) # needed to support naming the model
        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)

    def call(self, inputs): 
        #if ...
        input_wide, input_deep = inputs
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden1(norm_deep)
        # while len(hidden1)...
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([norm_wide, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        #return inverse(main_output).... Etc.

        return main_output, aux_output

In [ ]:
X_train_wide, X_train_deep, X_valid_wide, X_valid_deep, X_test_wide, X_test_deep = (
    X_train[:, :5],
    X_train[:, 2:],
    X_valid[:, :5],
    X_valid[:, 2:],
    X_test[:, :5],
    X_test[:, 2:],
)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss=("mse", "mse"), loss_weights=[0.9, 0.1], optimizer=optimizer,
              metrics=["RootMeanSquaredError"])

model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)

history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=10,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)))
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

# Saving the model and using Callbacks

In [10]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42
)

X_train_wide, X_train_deep, X_valid_wide, X_valid_deep, X_test_wide, X_test_deep = (
    X_train[:, :5],
    X_train[:, 2:],
    X_valid[:, :5],
    X_valid[:, 2:],
    X_test[:, :5],
    X_test[:, 2:],
)

X_new_wide, X_new_deep = X_test_wide[15:25], X_test_deep[15:25]

# Y_test[15:25]

In [ ]:
tf.random.set_seed(42)  
model = WideDeepModel(30, activation="relu", name="my_cool_model")
model.compile(loss=("mse", "mse"), loss_weights=[0.9, 0.1], optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=["RootMeanSquaredError"])

In [ ]:
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)

history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)))


model.save('hausing_widedeep_model', save_format='tf')



Use the saved model

In [14]:
model = tf.keras.models.load_model('hausing_widedeep_model')

y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))


y_pred_main, y_test[15:25]


1/1 [==============================] - 0s 53ms/step


2023-08-29 17:45:11.746733: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


(array([[1.3304222],
        [2.456015 ],
        [2.0427592],
        [2.227831 ],
        [1.1571676],
        [2.8501782],
        [2.5904763],
        [1.735521 ],
        [3.253336 ],
        [2.180563 ]], dtype=float32),
 array([1.362  , 1.784  , 1.875  , 1.398  , 1.375  , 4.25   , 4.056  ,
        1.388  , 5.00001, 1.621  ]))

In [ ]:
tensorboard --logdir
